In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan
from time import time
import numpy as np

In [3]:
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)

100%|██████████| 102/102 [00:12<00:00, 10.40it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


In [4]:
enc = ld_gan.utils.model_handler.load_model("xf_11111_s10l.py", 650, "enc")
gen = ld_gan.utils.model_handler.load_model("xf_11111_s10l.py", 650, "gen")
dis = ld_gan.utils.model_handler.load_model("xf_11111_s10l.py", 650, "dis")

/net/hciserver03/storage/oblum/venvs/compvisgpu02/lib/python2.7/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'ld_gan.models.enc.Enc' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/net/hciserver03/storage/oblum/venvs/compvisgpu02/lib/python2.7/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'ld_gan.models.gen.Gen' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


loaded model 'projects/xf_11111_s10l.py/model/e_0650.pth'
loaded model 'projects/xf_11111_s10l.py/model/g_0650.pth'
loaded model 'projects/xf_11111_s10l.py/model/d_0650.pth'


/net/hciserver03/storage/oblum/venvs/compvisgpu02/lib/python2.7/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'ld_gan.models.dis.Dis' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [5]:
z_enc = ld_gan.utils.model_handler.apply_model(enc, X)

In [20]:
s1 = ld_gan.sample.nn_sampler_life(enc, X, Y, 256)

In [21]:
t = time()
img_batch, y_batch, z_batch, z_batch = s1.next()
print time() - t

14.1738398075


In [22]:
s2 = ld_gan.sample.nn_sampler_life_2(enc, X, Y, 256)

In [23]:
t = time()
img_batch2, y_batch2, z_batch2, z_batch2 = s2.next()
print time() - t

1.68608999252


In [24]:
print np.all(img_batch == img_batch2)
print np.all(y_batch == y_batch2)
print np.all(z_batch == z_batch2)

True
True
True


In [1]:
import torch.nn.functional as F

In [22]:
batch_idxs = np.random.randint(0, len(z_enc), 256)

In [24]:
t = time()
from sklearn.neighbors import NearestNeighbors
dists, idxs = NearestNeighbors(n_neighbors=5,
                               n_jobs=10).fit(z_enc).kneighbors(z_enc[batch_idxs])
time() -t 

0.6632051467895508

In [25]:
import torch.nn.functional as F

In [28]:
z_enc[batch_idxs].shape

(256, 256)

In [29]:
z_enc.shape

(8189, 256)

In [18]:
batch_idxs = np.random.randint(0, len(z_enc), 256)

In [21]:
idxs.shape

(8189, 5)

In [15]:
batch_idxs = np.random.randint(0, len(z_enc), batch_size)

# get z_batch
batch_z_idxs = idxs[batch_idxs]
batch_z_all = z_enc[batch_z_idxs]
rand_weights = np.random.rand(n_neighbors, batch_size)
rand_weighst = rand_weights / np.sum(rand_weights, axis=0)
rand_weights = rand_weights.transpose()
z_batch = [np.average(z_all, 0, w) for w, z_all in zip(rand_weights, batch_z_all)]
z_batch = np.array(z_batch)

img_batch = imgs[batch_idxs]

y_batch = y[batch_idxs]

yield img_batch, y_batch, z_batch, z_batch

NameError: name 'np' is not defined

# sklearn stuff

In [10]:
import sklearn
import numpy as np
from sklearn.manifold import TSNE

In [5]:
from sklearn.utils import check_random_state
random_state = check_random_state(0)

In [12]:
n_components_original = 10
n_components_embedding = 2
X = random_state.randn(50, n_components_original).astype(np.float32)

In [13]:
tsne = TSNE(metric='manhattan', 
            n_components=n_components_embedding, 
            init=init, 
            random_state=0)

NameError: name 'init' is not defined